In [1]:
# Import libraries
import re
import nltk
import string
import pyforest
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# Read the CSV file
columns  = ["sentiment", "ID", "datetime", "query", "username", "text"]
df = pd.read_csv('sentiment_dataset.csv', delimiter=',', encoding = "ISO-8859-1", names = columns)
df.head()

<IPython.core.display.Javascript object>

,sentiment,ID,datetime,query,username,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
# Extract sentiment and text column, they will be relevant in this analysis
df = df[['sentiment','text']]
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [4]:
# # Fuction that performs cleaning using re library. 
# # It removes uppercase, brackets, links, punctuation etc
# def cleaning(a):
#     a = str(a).lower()
#     a = re.sub('\[.*?\]', '', a)
#     a = re.sub('[%s]' % re.escape(string.punctuation), '', a)
#     a = re.sub('\n', '', a)
#     a = re.sub('https?://\S+|www\.\S+', '', a)
#     a = re.sub('<.*?>+', '', a)
#     a = re.sub('\w*\d\w*', '', a)
#     return a

# df['text'] = df['text'].apply(cleaning)
# df.head()

In [5]:
# Removing the URl using re

def rem_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# Removing the stepwords 'english'

def rem_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stopwords.words('english'))])
    return text

# Removing the html 

def rem_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

#Removing the @

def remove_at_mentions(text):
    return re.sub(r'@\w+', '', text)


# Removing emojis by codes

def rem_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF" 
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# Fixing the text

def fixing(text):
    text = re.sub(r"Mr\'s", " he is", text)
    text = re.sub(r"Mr\'.", " he is", text)
    text = re.sub(r"Ms\'.", " She ", text)
    text = re.sub(r"She\'s", " she is", text)
    text = re.sub(r"I\'m", " i am", text)
    text = re.sub(r"haven\'t", " have not", text)
    text = re.sub(r"you\'r", " you are", text)
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    text = re.sub(r" plz", " please", text)
    text = re.sub(r"it\'s", " its", text) 
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"He\'s", " he is", text)
    text = re.sub(r"in\'s", " in", text)
    return text

# Seperate alphabets

def sep_alphabets(text):
    words = text
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)


def tweet_to_words(tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words ))

In [6]:
df['final_text'] = df['text']

df['final_text'] = df['final_text'].astype(str).apply(remove_at_mentions)
df['final_text'] = df['final_text'].apply(lambda x : rem_url(x))
df['final_text'] = df['final_text'].apply(lambda x : rem_stopwords(x))
df['final_text'] = df['final_text'].apply(lambda x : rem_html(x))
df['final_text'] = df['final_text'].apply(lambda x : rem_emoji(x))
df['final_text'] = df['final_text'].apply(lambda x : fixing(x))
df['final_text'] = df['final_text'].apply(lambda x : sep_alphabets(x))
df['final_text'] = df['final_text'].apply(lambda x : tweet_to_words(x))
df['final_text'] = df['final_text'].astype(str).replace("plz", "please", regex=True)

In [ ]:
# Checking for balance
df['sentiment'].value_counts()

In [ ]:
# # Removing stop words i.e., the, is, and, or, in, this etc
# s_words = stopwords.words('english')

# def removing(text):
#     a = ' '.join(i for i in text.split(' ') if i not in s_words)
#     return a
    
# df['text'] = df['text'].apply(removing)
# df.head()

In [ ]:
# Change 0 to negative and 4 to positive
class_dict = {0:'negative', 4:'positive'}
df['sentiment'] = df['sentiment'].apply(lambda x:  class_dict[x])
df.head()

In [ ]:
# # Stemming words that have same meaning
# stemmer = nltk.SnowballStemmer("english")

# def stemming(text):
#     text = ' '.join(stemmer.stem(i) for i in text.split(' '))
#     return text

# df['text'] = df['text'].apply(stemming)
# df.head()

In [ ]:
# Split the data to train data and test data
X = df['text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Using Logistic Regression
clf = LogisticRegression(penalty="l2", random_state=42, C=0.01)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Training Accuracy:", clf.score(X_train, y_train))
print("Testing Accuracy:", clf.score(X_test, y_pred))